This notebook was automatically generated from your MAST-ML run so you can recreate the
plots. Some things are a bit different from the usual way of creating plots - we are
using the [object oriented
interface](https://matplotlib.org/tutorials/introductory/lifecycle.html) instead of
pyplot to create the `fig` and `ax` instances.


In [ ]:
"""
This module contains a collection of functions which make plots (saved as png files) using matplotlib, generated from
some model fits and cross-validation evaluation within a MAST-ML run.

This module also contains a method to create python notebooks containing plotted data and the relevant source code from
this module, to enable the user to make their own modifications to the created plots in a straightforward way (useful for
tweaking plots for a presentation or publication).
"""

import math
import os
import pandas as pd
import itertools
import warnings
import logging
from collections import Iterable
from os.path import join
from collections import OrderedDict

# Ignore the harmless warning about the gelsd driver on mac.
warnings.filterwarnings(action="ignore", module="scipy",
                        message="^internal gelsd")
# Ignore matplotlib deprecation warning (set as all warnings for now)
warnings.filterwarnings(action="ignore")

import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve

import matplotlib
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure, figaspect
from matplotlib.animation import FuncAnimation
from matplotlib.font_manager import FontProperties
import matplotlib.mlab as mlab
from scipy.stats import gaussian_kde
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes

# Needed imports for ipynb_maker
from mastml.utils import nice_range
from mastml.metrics import nice_names

import inspect
import textwrap
from pandas import DataFrame, Series

import nbformat

from functools import wraps

matplotlib.rc('font', size=18, family='sans-serif') # set all font to bigger
matplotlib.rc('figure', autolayout=True) # turn on autolayout

# adding dpi as a constant global so it can be changed later
DPI = 250

log = logging.getLogger() # only used inside ipynb_maker I guess



In [ ]:
def stat_to_string(name, value):
    """
    Method that converts a metric object into a string for displaying on a plot

    Args:

        name: (str), long name of a stat metric or quantity

        value: (float), value of the metric or quantity

    Return:

        (str), a string of the metric name, adjusted to look nicer for inclusion on a plot

    """

    " Stringifies the name value pair for display within a plot "
    if name in nice_names:
        name = nice_names[name]
    else:
        name = name.replace('_', ' ')

    # has a name only
    if not value:
        return name
    # has a mean and std
    if isinstance(value, tuple):
        mean, std = value
        return f'{name}:' + '\n\t' + f'{mean:.3f}' + r'$\pm$' + f'{std:.3f}'
    # has a name and value only
    if isinstance(value, int) or (isinstance(value, float) and value%1 == 0):
        return f'{name}: {int(value)}'
    if isinstance(value, float):
        return f'{name}: {value:.3f}'
    return f'{name}: {value}' # probably a string


def plot_stats(fig, stats, x_align=0.65, y_align=0.90, font_dict=dict(), fontsize=14):
    """
    Method that prints stats onto the plot. Goes off screen if they are too long or too many in number.

    Args:

        fig: (matplotlib figure object), a matplotlib figure object

        stats: (dict), dict of statistics to be included with a plot

        x_align: (float), float denoting x position of where to align display of stats on a plot

        y_align: (float), float denoting y position of where to align display of stats on a plot

        font_dict: (dict), dict of matplotlib font options to alter display of stats on plot

        fontsize: (int), the fontsize of stats to display on plot

    Returns:

        None

    """

    stat_str = '\n'.join(stat_to_string(name, value)
                           for name,value in stats.items())

    fig.text(x_align, y_align, stat_str,
             verticalalignment='top', wrap=True, fontdict=font_dict, fontproperties=FontProperties(size=fontsize))


def make_fig_ax(aspect_ratio=0.5, x_align=0.65, left=0.10):
    """
    Method to make matplotlib figure and axes objects. Using Object Oriented interface from https://matplotlib.org/gallery/api/agg_oo_sgskip.html

    Args:

        aspect_ratio: (float), aspect ratio for figure and axes creation

        x_align: (float), x position to draw edge of figure. Needed so can display stats alongside plot

        left: (float), the leftmost position to draw edge of figure

    Returns:

        fig: (matplotlib fig object), a matplotlib figure object with the specified aspect ratio

        ax: (matplotlib ax object), a matplotlib axes object with the specified aspect ratio

    """
    # Set image aspect ratio:
    w, h = figaspect(aspect_ratio)
    fig = Figure(figsize=(w,h))
    FigureCanvas(fig)

    # Set custom positioning, see this guide for more details:
    # https://python4astronomers.github.io/plotting/advanced.html
    #left   = 0.10
    bottom = 0.15
    right  = 0.01
    top    = 0.05
    width = x_align - left - right
    height = 1 - bottom - top
    ax = fig.add_axes((left, bottom, width, height), frameon=True)
    fig.set_tight_layout(False)
    
    return fig, ax


In [ ]:
def plot_residuals_histogram(y_true, y_pred, savepath,
                             stats, title='residuals histogram', label='residuals'):
    """
    Method to calculate and plot the histogram of residuals from regression model

    Args:

        y_true: (numpy array), array of true y data values

        y_pred: (numpy array), array of predicted y data values

        savepath: (str), path to save the plotted precision-recall curve

        stats: (dict), dict of training or testing statistics for a particular run

        title: (str), title of residuals histogram

        label: (str), label used for axis labeling

    Returns:

        None

    """

    # make fig and ax, use x_align when placing text so things don't overlap
    x_align = 0.64
    fig, ax = make_fig_ax(x_align=x_align)

    #ax.set_title(title)
    # do the actual plotting
    residuals = y_true - y_pred

    #Output residuals data and stats to spreadsheet
    path = os.path.dirname(savepath)
    pd.DataFrame(residuals).describe().to_csv(os.path.join(path,'residual_statistics.csv'))
    pd.DataFrame(residuals).to_csv(path+'/'+'residuals.csv')

    #Get num_bins using smarter method
    num_bins = get_histogram_bins(y_df=residuals)
    ax.hist(residuals, bins=num_bins, color='b', edgecolor='k')

    # normal text stuff
    ax.set_xlabel('Value of '+label, fontsize=16)
    ax.set_ylabel('Number of occurences', fontsize=16)

    # make y axis ints, because it is discrete
    #ax.yaxis.set_major_locator(MaxNLocator(integer=True))

    plot_stats(fig, stats, x_align=x_align, y_align=0.90)
    plot_stats(fig, pd.DataFrame(residuals).describe().to_dict()[0], x_align=x_align, y_align=0.60)

    fig.savefig(savepath, dpi=DPI, bbox_inches='tight')


In [ ]:
from numpy import array
from collections import OrderedDict
from io import StringIO
y_true = array([2.45647989e+02, 2.19470531e+02, 1.93721694e+02, 2.53584855e+02,
       2.44558934e+02, 7.98843255e+01, 1.43144270e+02, 1.40791488e+02,
       1.14194239e+02, 9.53431111e+01, 2.14252916e+02, 1.76193353e+02,
       1.11989369e+02, 1.13270942e+02, 1.21012896e+02, 1.17557012e+02,
       1.01679243e+02, 1.81776188e+02, 1.71679718e+02, 1.95159840e+02,
       1.90460889e+02, 2.00504538e+02, 1.25081235e+02, 1.37316410e+02,
       1.25772961e+02, 3.35551879e+02, 2.09882629e+02, 7.06664565e+01,
       7.23673571e+01, 8.84608475e+01, 8.09403347e+01, 1.87519625e+02,
       1.58367410e+02, 1.21204438e+02, 7.02166732e+01, 2.35011420e+01,
       1.18303763e+02, 1.71608093e+02, 1.21368422e+02, 2.70007913e+02,
       2.91384616e+02, 6.60766101e+01, 1.11442294e+02, 1.00971120e+02,
       2.36664682e+02, 4.08488586e+02, 1.22389663e+02, 2.72804393e+02,
       7.86712295e+01, 8.81149314e+01, 8.42025056e+01, 9.25437335e+01,
       8.95029615e+01, 1.02036436e+02, 1.82740912e+02, 6.43729766e+02,
       2.94591088e+01, 0.00000000e+00, 7.58782445e+01, 4.93432857e+01,
       1.58218632e+02, 3.63631922e+01, 0.00000000e+00, 2.61724803e+02,
       7.45861041e+01, 0.00000000e+00, 7.93342932e+01, 7.03541024e+01,
       1.12003551e+02, 1.98106664e+02, 2.18946397e+02, 2.02689680e+02,
       1.50472708e+02, 5.96349894e+01, 7.08357719e+01, 0.00000000e+00,
       0.00000000e+00, 4.31928159e+01, 3.44706557e+01, 4.89115430e+01,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 5.93437233e-01, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       8.92356506e+01, 2.68282539e+01, 1.74713200e+00, 0.00000000e+00,
       6.03605982e+01, 6.37138607e+00, 3.27899307e+01, 5.38425541e+01,
       0.00000000e+00, 6.50312714e+01, 4.34882312e+01, 2.67022126e+01,
       4.39956935e+01, 0.00000000e+00, 0.00000000e+00, 1.92047597e+01,
       4.42073704e-01, 0.00000000e+00, 1.33559658e+02, 1.55792869e+02,
       0.00000000e+00, 3.02287500e+01, 0.00000000e+00, 0.00000000e+00,
       4.93632300e+01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.62043592e+01, 1.48100516e+01, 1.38046084e+01,
       6.06802766e+01, 3.45198214e+01, 1.17125778e+02, 2.88123722e+01,
       1.22038739e+02, 3.01702447e+01, 5.38520339e+01, 3.32000681e+01,
       0.00000000e+00, 2.36626028e+01, 3.13780640e+01, 3.66852424e+01,
       0.00000000e+00, 0.00000000e+00, 3.62570480e+01, 1.37408713e+02,
       7.80094958e+01, 1.00859781e+02, 5.86508558e+01, 0.00000000e+00,
       2.24784003e+01, 1.51553671e+02, 0.00000000e+00, 8.69178900e+00,
       3.29412986e+01, 4.36717688e+00, 2.76028721e+02, 2.63839575e+02,
       9.97625282e+01, 1.85524542e+02, 2.80876470e+02, 2.02664553e+02,
       1.26777344e+02, 2.30921905e+02, 1.34506442e+02, 3.21989575e+02,
       5.08392287e+01, 4.74586631e+01, 3.50768896e+01, 2.23740347e+01,
       2.91802070e+01, 2.93008151e+01, 0.00000000e+00, 1.21655324e+01,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 5.88853197e+01,
       7.81817223e+01, 0.00000000e+00, 4.48488874e+01, 4.49236078e+01,
       4.83755926e+01, 1.70719435e+01, 0.00000000e+00, 4.75455355e+01,
       5.40734041e+01, 6.98113911e+01, 6.77439450e+01, 6.45192863e+01,
       7.45637353e+01, 6.66463830e+01, 4.63304068e+01, 4.31541225e+01,
       3.74738040e+01, 5.83323071e+01, 5.13090164e+01, 0.00000000e+00,
       5.33902792e+01, 3.42510607e+01, 3.13514481e+01, 1.62892176e+02,
       2.14364169e+02, 2.24402165e+02, 1.54697467e+02, 1.22980240e+02,
       1.24729865e+02, 1.28386129e+02, 1.34671642e+02, 1.02151271e+02,
       8.15909270e+01, 0.00000000e+00, 4.31287284e+01, 4.95815034e+01,
       2.44136176e+01, 1.02395743e+02, 1.03527711e+01, 2.87242187e+01,
       5.17559553e+01, 1.40515010e+02, 5.19507218e+01, 7.16777317e+01,
       1.01745118e+01, 1.15904985e+01, 2.18195920e+01, 5.50100622e+01,
       0.00000000e+00, 5.76088495e+01, 8.58908577e+01, 2.11232453e+01,
       0.00000000e+00, 0.00000000e+00, 5.31766390e-01, 1.12780820e+01,
       2.61974495e+01, 0.00000000e+00, 3.56929084e+01, 4.21875225e+01,
       1.42333954e+02, 1.57242981e+02, 2.02269808e+02, 2.65270592e+02,
       1.93229456e+02, 2.86026589e+02, 1.57972803e+02, 3.78453371e+02,
       2.87661839e+02, 8.83272115e+01, 9.96813433e+01, 1.87875800e+02,
       1.31761462e+02, 1.29607780e+02, 1.53246503e+02, 1.11920471e+02,
       1.14498162e+02, 1.30390352e+02, 1.03756755e+02, 1.00207954e+02,
       2.35409225e+02, 2.92390663e+02, 1.69625113e+02, 1.23246629e+02,
       1.22677657e+02, 1.06220359e+02, 2.12302508e+02, 1.78586168e+02,
       1.22431670e+02, 1.21428977e+02, 1.69573331e+02, 3.57145351e+02,
       2.62896301e+02, 1.91512363e+02, 2.21055904e+02, 2.13493682e+02,
       1.87894409e+02, 7.69784095e+01, 1.80660958e+02, 1.10209088e+02,
       2.08659157e+02, 1.98592674e+02, 1.00117427e+02, 1.87885255e+02,
       9.44335089e+01, 1.88513164e+02, 9.48376021e+01, 1.21127399e+02,
       3.54737752e+02, 2.59216571e+02, 2.38522270e+02, 3.30939913e+02,
       1.42826192e+02, 1.04459645e+02, 7.99859482e+01, 1.34599602e+02,
       1.82220981e+02, 2.13365657e+02, 9.73721500e+01, 1.18314107e+02,
       1.39928045e+02, 6.45557199e+01, 1.43519281e+02, 1.93517827e+02,
       1.48340548e+02, 1.91404536e+02, 2.51703483e+02, 3.47295901e+02,
       1.42416121e+02, 1.10472455e+02, 9.57786406e+01, 6.77164974e+01,
       8.46282081e+01, 1.43722366e+02, 1.94691060e+02, 1.51089123e+02,
       1.22185191e+02, 1.10480194e+02, 2.91372971e+01, 3.51009606e+01,
       7.09524931e+01, 1.24273117e+02, 1.27664645e+02, 6.88773537e+01,
       3.31838908e+01, 7.58528445e+01, 4.88429076e+01, 1.33635430e+02,
       2.05075217e+02, 1.62590223e+02, 1.21553577e+02, 1.13527293e+02,
       1.85414463e+02, 2.18074203e+02, 1.22370956e+02, 1.63982208e+02,
       1.24542111e+02, 1.23935967e+02, 1.74060277e+02, 1.13060644e+02,
       8.66997511e+01, 1.10285836e+02, 1.83271284e+02, 1.22316043e+02,
       1.17247059e+02, 9.79185800e+01, 1.04244208e+02, 1.84901449e+02,
       1.84591316e+02, 1.83151200e+02, 1.56027039e+02, 1.17143169e+02,
       1.09297300e+02, 9.87844184e+01, 1.35955483e+02, 1.62419081e+02,
       1.38582903e+02, 1.63497124e+02, 8.86858956e+01, 1.82613120e+02,
       3.33198729e+01, 2.86171985e+01, 2.15562064e+01, 2.00619416e+01,
       1.09582964e+02, 1.13170481e+02, 7.26752642e+01, 6.16998797e+01,
       1.05859697e+02, 9.98669764e+01, 3.41670362e+02, 0.00000000e+00,
       1.09972952e+02, 8.70884029e+01, 9.43179777e+01, 1.32019981e+02,
       1.29167628e+02, 9.72556960e+01, 0.00000000e+00, 9.82564416e+01,
       0.00000000e+00, 1.12174952e+02, 7.18885960e+01, 5.63388158e+01,
       5.63192894e+01, 4.94793928e+01, 6.24481656e+01, 2.83696735e+01,
       4.87758689e+01, 0.00000000e+00, 1.10177861e+01, 7.99376675e+00,
       0.00000000e+00, 2.35812529e+01, 1.20026452e+02, 2.71292986e+01,
       0.00000000e+00, 3.40153297e+01, 4.92018258e+01, 2.67854426e+02,
       2.07757183e+02, 7.51456208e+01, 8.29965720e+01, 1.52154945e+02,
       9.16707430e+01, 6.94738840e+01, 9.11059876e+01, 3.91108879e+01,
       4.69507504e+01, 5.25504455e+01, 5.66656498e+01, 3.88036349e+01,
       1.52570887e+02, 8.42613186e+01, 1.23233184e+02, 9.01656399e+01,
       3.15229082e+01, 3.15042600e+01, 3.74221989e+01, 2.93690973e+01,
       3.66110447e+01, 5.40260430e+01, 1.20061041e+02, 5.36862536e+01,
       2.79073032e+01, 1.99708105e+01, 1.73113038e+02, 7.17549475e+01,
       0.00000000e+00, 2.89029515e+02, 0.00000000e+00, 3.35261904e+01,
       4.72079029e+01])
y_pred = array([0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       3.81945241e-199, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 2.33035113e-316,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000])
savepath = '/home/nerve/Desktop/skunkworks/lane_schultz/hyper_parameters/test/DoNothing/DoNothing/KernelRidge_DoNothing_DoNothing_GridSearch/RepeatedKFold/split_6/test_residuals_histogram.png'
stats = OrderedDict([('R2', 1.0), ('root_mean_squared_error', 132.04314402748628), ('mean_absolute_error', 99.82208830197554), ('rmse_over_stdev', 1.3549567913109741)])
title = 'test_residuals_histogram'
label = 'Energy above convex hull (meV/atom)'

In [ ]:
import pandas as pd
from IPython.display import Image, display

plot_residuals_histogram(y_true, y_pred, savepath, stats, title, label)
display(Image(filename='test_residuals_histogram.png'))
